In [8]:
import pandas as pd
import os
from dbHelpers import createEngine
import uuid

In [9]:
engine = createEngine()

In [ ]:
path = '../scratch/halloween_data'

In [18]:
fips_state_df = pd.read_sql('select * from npd.fips_state', con = engine)
fips_state_df.set_index('abbreviation', inplace=True)

In [22]:
def show_or_load(df, table_name, schema_name, load=False):
    if load:
        df.to_sql(table_name, schema = schema_name, con = engine)
    else:
        df.head()

In [23]:
primary_to_bool = {1: True, 0: False}


def convertBool(val):
    if val in primary_to_bool.keys():
        return primary_to_bool[val]
    else:
        return False

In [ ]:
df_dict={}
for f in os.listdir(path):
    if '.csv' in f:
        tablename = f.split('.csv')[0]
        df = pd.read_csv(os.path.join(path,f))
        df_dict[f]=df
        df, tablename, index=False, schema = 'raw_csv', con = engine, if_exists='replace')

/var/folders/zt/jvxrh15j40sfz4x_sldwpck00000gp/T/ipykernel_11117/2424709856.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(path,f))


In [ ]:
practitioner_df = df_dict['practitioner.csv']
#note: we can do this because each practitioner only appears once in this table
practitioner_df['id'] = [uuid.uuid4() for i in practitioner_df.index]
practitioner_df.rename(columns = {'gender_code': 'sex', 'name_prefix': 'prefix', 'name_suffix': 'suffix'}, inplace=True)
npi1 = practitioner_df[['npi']]
npi1['entity_type_code'] = 1
practitioner_taxonomy_df = df_dict['practitionerrole.csv']
merged_taxonomy_df = practitioner_taxonomy_df.merge(practitioner_df, left_on = 'practitioner_id', right_on = 'npi', suffixes = ('tax', 'individual')) 
merged_taxonomy_df = merged_taxonomy_df.loc[merged_taxonomy_df['state_code']!='ZZ']
merged_taxonomy_df['state_code'] = merged_taxonomy_df['state_code'].apply(lambda x: fips_state_df.loc[x]['id'])
merged_taxonomy_df.rename(columns={'idindividual': 'individual_id', 'taxonomy_code':'nucc_code'}, inplace=True)
provider_to_taxonomy_df = merged_taxonomy_df[['individual_id', 'nucc_code', 'is_primary']]
provider_to_taxonomy_df['is_primary'] = provider_to_taxonomy_df['is_primary'].apply(lambda x: convertBool(x))
dedup_taxonomy_df = provider_to_taxonomy_df.sort_values(by='is_primary', ascending=False)[
        ['individual_id', 'nucc_code', 'is_primary']].drop_duplicates(subset=['nucc_code', 'individual_id'])
dedup_taxonomy_df['id'] = [uuid.uuid4() for i in dedup_taxonomy_df.index]
credential_df = provider_to_taxonomy_df.merge(dedup_taxonomy_df, on = ['individual_id', 'nucc_code'], suffixes = ('tax', 'cred'))
credential_df.rename(columns={'idtax': 'provider_to_taxonomy_id'}, inplace=True)

/var/folders/zt/jvxrh15j40sfz4x_sldwpck00000gp/T/ipykernel_11117/2346510260.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  npi1['entity_type_code'] = 1


KeyError: "['individual_id'] not in index"

In [32]:
merged_taxonomy_df.head()

,idtax,practitioner_id,nucc_code,state_code,license_number,is_primary,npi,last_name,first_name,middle_name,prefix,suffix,credential_text,is_sole_proprietor,sex,13872713232,idindividual
0,3,1487102463,103T00000X,20,2744,0,1487102463,FINLEY,MARIANNA,NaN,NaN,NaN,LPC,True,F,31c1b247-83da-440a-bc7a-2673f1591bcd,3723eac5-b768-4444-bd71-c7bbcc1aba4f
1,11,1033667035,2255A2300X,42,RT006860,1,1033667035,OXLEY,MEGAN,FRANCES,MS.,NaN,"M.ED., LAT, ATC",True,F,bcf58541-7423-4597-8a03-409ddfba52a5,b3e06dd5-a15b-49f4-a0e7-d630638841ad
2,50,1225586258,207Q00000X,41,PA190185,1,1225586258,SISNEROS,CONTESSA,NaN,NaN,NaN,PA-C,True,F,83241ffe-6823-4ba1-812b-2e88ca3bc8e8,8f61bd63-b033-4feb-a628-ed9cd87c260e
3,97,1790233658,208100000X,26,5501017928,1,1790233658,SCHLEMMER,AMBER,M,NaN,NaN,DPT,True,F,faa6fa37-e51d-4cf8-86e1-134775d16ba3,9d1a3887-2482-410f-a4db-d5a2378d9abc
4,106,1316495229,174H00000X,13,2013-211,1,1316495229,TERRY,KATIE,NaN,NaN,NaN,CLD,True,F,c5e07e17-22cc-4c57-9150-10d85390e296,6a281cb1-a4e3-4403-b04a-806c5163de1e


In [ ]:
organization_df = df_dict['organization.csv']
organization_df['is_primary'] = True
organization_df.rename(columns={'id':'old_org_id', 'parent_id':'old_parent_id'}, inplace=True)
organization_df['org_id'] = [uuid.uuid4() for i in organization_df.index]
organization_df.set_index('old_org_id', inplace=True)
organization_df['org_parent_id'] = [organization_df['old_parent_id'].apply(lambda x: organization_df.loc[x]['org_id'])]
organization_npi_df = df_dict['organization_npi.csv']
organization_npi_df.rename(columns={'organization_id':'old_org_id'}, inplace=True)
organization_npi_df['id'] = [uuid.uuid4() for i in organization_npi_df.index]
npi2 = organization_npi_df[['npi']]
npi2['entity_type_code'] = 2
merged_organization_df = organization_npi_df.merge(organization_df, on='old_org_id')
merged_organization_df.rename(columns={'org_id':'parent_id'}, inplace=True)
organization_df.rename(columns = {'org_id':'id', 'org_parent_id': 'parent_id'}, inplace=True)


NameError: name 'organizaiton_df' is not defined

In [ ]:
npi_df = pd.concat([npi1,npi2])

In [ ]:
endpoint_df = df_dict['endpoint.csv']
endpoint_df.rename(columns={'id':'endpoint_id'})
ehr_vendor_df = endpoint_df[['vendor_name']].drop_duplicates()
ehr_vendor_df['id'] = [uuid.uuid4() for i in ehr_vendor_df.index]
ehr_vendor_df.rename(columns={'vendor_name':'name'}, inplace=True)
ehr_vendor_df.set_index('vendor_name', inplace=True)
endpoint_df['ehr_vendor_id'] = endpoint_df['vendor_name'].apply(lambda x: ehr_vendor_df.loc[x]['id'])
endpoint_df['environment_type_id'] = 'prod'
endpoint_df['endpoint_connection_type_id'] = 'hl7-fhir-rest'
endpoint_df['id'] = [uuid.uuid4() for i in endpoint_df.index]
endpoint_df.rename(columns={'fhir_url':'address'})

In [ ]:
org_to_endpoint_df = df_dict['organization_endpoint.csv']
merged_org_to_endpoint_df = org_to_endpoint_df.merge(endpoint_df, on = 'endpoint_id').merge(organization_npi_df, left_on = 'organization_npi', right_on = 'npi', suffix = ('endpoint', 'organization'))
merged_org_to_endpoint_df= merged_org_to_endpoint_df[['idendpoint', 'idorganization']].rename(columns = {'idendpoint': 'endpoint_instance_id', 'idorganization':'organization_id'})

In [ ]:
address_df = df_dict['location.csv']
address_df.rename(columns={'id':'address_us_id', 'line':'delivery_line_1', 'postalcode':'zipcode'}, inplace=True)
address_df['id']= [uuid.uuid4() for i in address_df.index]
address_df['state_code'] = address_df['state'].apply(lambda x: fips_state_df.loc[x]['id'])
location_npi_df = df_dict['location_npi.csv']
merged_location_df = location_npi_df.merge(address_df, left_on='address_us_id', right_on = 'location_id').merge(npi_df,practitioner_df,organization_df, on = 'npi', suffixes=('npi','individual','organization'))
merged_location_df.rename(columns={'id':'address_id', 'idindividual':'individual_id', 'idorganization':'organization_id'}, inplace=True)
merged_location_df['address_use_id'] = 2
individual_to_address_df = merged_location_df[['address_id','individual_id', 'address_use_id']].dropna(how='any')
location_df = merged_location_df[['address_id','organization_id','name', 'address_use_id']].dropna(how='any')
location_df['id'] = [uuid.uuid4() for i in location_df.index]
location_to_endpoint_df = location_df.merge(org_to_endpoint_df, on = 'organization_id')[['id', 'endpoint_id']].dropna(how = 'any').rename(columns = {'id':'location_id'})


In [ ]:
provider_to_organization_df = df_dict['personal_npi_to_organizational_npi.csv']
merged_provider_to_org_df = provider_to_organization_df.merge(practitioner_df, left_on = 'personal_npi', right_on = 'npi').merge(merged_organization_df, left_on = 'organizational_npi', right_on = 'npi', suffixes = ('individual', 'organization'))
merged_provider_to_org_df.rename(columns = {'idindividual':'individual_id', 'idorganization':'organization_id'}, inplace=True)
merged_provider_to_org_df['id'] = [uuid.uuid4() for i in merged_provider_to_org_df.index]
merged_provider_to_org_df['relationship_type_id'] = 2
provider_to_location_df = merged_provider_to_org_df.merge(location_df, on='organization_id')
provider_to_location_df['id'] = [uuid.uuid4() for i in provider_to_location_df.index]

In [ ]:
schema_name = 'npd'
load = False

# load npi
show_or_load(npi_df, 'npi', schema_name, load)

# load individual
show_or_load(practitioner_df[['id', 'sex']], 'individual', schema_name, load)

practitioner_df.rename(columns={'id':'individual_id'}, inplace=True)

# load individual_to_name
show_or_load(practitioner_df[['individual_id', 'first_name', 'middle_name', 'last_name', 'prefix', 'suffix']], 'individual', schema_name, load)

# load provider
show_or_load(practitioner_df[['npi', 'individual_id']], 'provider', schema_name, load)

# load organization
show_or_load(organization_df[['id', 'parent_id']], 'organization', schema_name, load)
show_or_load(merged_organization_df[['id', 'parent_id']], 'organization', schema_name, load)

organization_df.rename(columns={'id':'organization_id'}, inplace=True)
merged_organization_df.rename(columns={'id':'organization_id'}, inplace=True)

# load organization_to_name
show_or_load(organization_df[['organization_id', 'name', 'is_primary']], 'organization_to_name', schema_name, load)
show_or_load(merged_organization_df[['organization_id', 'name', 'is_primary']], 'organization_to_name', schema_name, load)

# load clinical_organization
show_or_load(merged_organization_df[['organization_id', 'npi']], 'clinical_organization', schema='npd', con = engine)

# load ehr_vendor
show_or_load(ehr_vendor_df[['id', 'name']], 'ehr_vendor', schema_name, load)

# load endpoint_instance
show_or_load(endpoint_df[['id', 'ehr_vendor_id', 'address', 'endpoint_connection_type_id', 'environment_type_id']], 'endpoint_instance', schema_name, load)

# load address_us
show_or_load(location_df[['address_us_id', 'delivery_line_1','city','state_code','zipcode']].rename(columns={'address_us_id':'id'}), 'address_us', schema_name, load)

# load address
show_or_load(location_df[['id', 'address_us_id']], 'address', schema_name, load)

# load individual_to_address
show_or_load(individual_to_address_df, 'individual_to_address', schema_name, load)

# load organization_to_address
show_or_load(location_df[['address_id','organization_id']], 'organization_to_address', schema_name, load)

# load location
show_or_load(location_df[['id','address_id','organization_id']], 'location', schema_name, load)

# load location_to_endpoint
show_or_load(location_to_endpoint_df, 'location_to_endpoint', schema_name, load)

# load provider_to_organization
show_or_load(merged_provider_to_org_df.dropna(how='any'), 'provider_to_organization', schema_name, load)

# load provider_to_location
show_or_load(provider_to_location_df.dropna(how='any'), 'provider_to_location', schema_name, load)

# load provider_to_taxonomy
show_or_load(dedup_taxonomy_df, 'provider_to_taxonomy', schema_name, load)

# load provider_to_credential
show_or_load(credential_df[['license_number', 'state_code', 'provider_to_taxonomy_id']], 'provider_to_taxonomy', schema_name, load)